In [11]:
import sys
sys.path.append('..')

import numpy as np
from pytenet.hartree_fock_mps import generate_single_state
from pytenet.hamiltonian_thc import eval_func, generate_thc_mpos_by_layer_qn, get_t, get_h1_spin, get_g_spin
from pytenet.global_krylov_method import generate_krylov_space_in_disk, get_W, get_S, remain_only_tridiagonal_elements
from pytenet.global_krylov_method import generate_Hamiltonian_with_occupation_number, generate_reduced_H_non_ortho
import numpy as np
from scipy import sparse
import copy
import h5py
from numpy.linalg import norm
#np.set_printoptions(precision=4,suppress=True)
import scipy.io
import matplotlib.pyplot as plt
import pickle
import pytenet as ptn


Load and initialize datas: 

no is number of spatial orbitals

L is number of spatial orbitals, L = no

We DON'T need t_spin, g_spin now, only 'original' datas are enough 

r_THC is THC rank

In [12]:
#load integrals
with h5py.File("../data_water/eri_water.hdf5", "r") as f:
#with h5py.File("/work_fast/ge49cag/pytenet_yu/water/eri_water.hdf5", "r") as f:
    eri = f["eri"][()]
    hkin = f["hkin"][()]
    hnuc = f["hnuc"][()]

#print(np.linalg.norm(eri))
#print(eri.shape)

no = eri.shape[0]
MV = eri.reshape(no*no,no*no)

u = np.load("../data_water/x.npy")
#u = np.load("/work_fast/ge49cag/pytenet_yu/water/x.npy")
X_mo = u.transpose(1,0)
g_thc, Z_mo = eval_func(u,eri,hkin,hnuc,)
h1 = hnuc+hkin
nmo = X_mo.shape[1]
L = X_mo.shape[1]
g_thc = g_thc.reshape(nmo, nmo, nmo, nmo)
r_thc = X_mo.shape[0]

7
(7, 28)
(28, 28)
rl errV: 2.8386751875274264e-12
abs errV: 2.0615721155266396e-11
errt: 7.097049412242525e-13
errh: 2.585427402664151e-13
errht: 9.079449636842276e-14


These Hamiltonian are exact molecular Hamiltonian and molecular Hamiltonian reconstructed by THC tensors. The calculation cost time, so that we store them in disk and load them when needed. For water molecule H2O in STO-6G basis, the error is small for r_THC = 28.

Actually, considering there are always 10 electrons for a water molecule, we only retain the elements which operator quantum states with 10 electrons.

In [13]:
#load Hamiltonian generated by above coefficients
# H_correct = scipy.io.mmread('data_water/H_water_correct.mtx').tocsr()
# H_correct_10e = generate_Hamiltonian_with_occupation_number(H_correct.real, 10)
# e, v = sparse.linalg.eigsh(H_correct_10e, which = 'SA', k = 10)
# e_ground = e[0]
# e_1st_ex = e[1]

e_ground = -84.92262983120877
e_1st_ex = -84.52780562388604
e_2nd_ex = -84.46820215626565
e_3rd_ex = -84.42486181064207
e_4th_ex = -84.42379006099287

To construct 'original mpo' of molecular Hamiltonian, currently we can use ptn.hamiltonian.spin_molecular_hamiltonian_mpo(h1, g_phy), which consider the spins automatically, we don't need generating spin-version datas anymore. The result of ptn.hamiltonian.spin_molecular_hamiltonian_mpo(h1, g_phy) is mpo whose physical dimension is 4.

In [14]:
# h1_spin = get_h1_spin(h1)
# g_spin = get_g_spin(eri)
g_phy =  eri.transpose(0, 2, 1, 3)
#mpo_ref = ptn.hamiltonian.molecular_hamiltonian_mpo(h1_spin, g_spin_phy)
mpo_ref = ptn.hamiltonian.spin_molecular_hamiltonian_mpo(h1, g_phy)
print(mpo_ref.bond_dims)

[1, 16, 58, 96, 96, 58, 16, 1]


Generate Hartree-Fock state. See function generate_single_state for further details.

In [15]:
HFS = generate_single_state(7, [3, 3, 3, 3, 3, 0, 0])

Generate THC-MPO by layers, using THC tensors. 
It returns a list of H_mu_nu, each H_mu_nu is also a list, which contains four smaller MPOs with bond dims 2.
The final element of this list is MPO for kinetic term.
The physical bond_dim is 4.

In [16]:
#generate thc_mpo
t = get_t(h1, eri)
H_mu_nu_list_spin_layer = generate_thc_mpos_by_layer_qn(X_mo, Z_mo, L, t)

print(type(H_mu_nu_list_spin_layer))
print(type(H_mu_nu_list_spin_layer[0]))
print(type(H_mu_nu_list_spin_layer[0][0]))
print((H_mu_nu_list_spin_layer[0][0].bond_dims))

<class 'list'>
<class 'list'>
<class 'pytenet.mpo.MPO'>
[1, 2, 2, 2, 2, 2, 2, 1]


For ground state finding, we use Hatree fock state |11111111110000> as initial state.

In [17]:
#excited states for different molecules could have different total spins, please try different spin sections.
#for water in sto-6g: 1st excited state has spin 1
HFS = generate_single_state(7, [3, 3, 3, 3, 3, 0, 0])
#HFS.orthonormalize('left')
#HFS.orthonormalize('right')


We generate a group of orthogonal Krylov vectors using THC-MPO, with bond dim 40 for Krylov vectors. The vectors are stored in the folder = 'foldername', thus you don't have to generate them again for next time use. 

In [18]:
N_Krylov = 40
psi_original = copy.deepcopy(HFS)
max_bond_Krylov = 80
#max_bond_Krylov = 75
trunc_tol = 0
foldername = f"../water_Krylov"
#Krylov vectors are included in data, you dont have to run generate it. ofc, you can -regenerate it to verify the algorithm using the following code:
generate_krylov_space_in_disk(N_Krylov, H_mu_nu_list_spin_layer, psi_original, max_bond_Krylov, trunc_tol, r_thc, foldername)

#it indicates that even though during the calculation the bond dims exceed 40, but we only need 37 for Krylov vectors.

[1, 4, 11, 13, 15, 11, 4, 1]
2
[1, 4, 16, 31, 31, 16, 4, 1]
3
[1, 4, 16, 31, 31, 16, 4, 1]
4
[1, 4, 16, 31, 31, 16, 4, 1]
5
[1, 4, 16, 31, 31, 16, 4, 1]
6
[1, 4, 16, 31, 31, 16, 4, 1]
7
[1, 4, 16, 31, 31, 16, 4, 1]
8
[1, 4, 16, 31, 31, 16, 4, 1]
9
[1, 4, 16, 31, 31, 16, 4, 1]
10
[1, 4, 16, 31, 31, 16, 4, 1]
11


Make use of method proposed in https://journals.aps.org/prb/abstract/10.1103/PhysRevB.85.205119 to improve the orthogonality of Krylov vectors. 

In [ ]:
N_use = 40
# C = coeff_canonical_orthogonalization(N_use, foldername)
# vector_list = generate_re_ortho_space_with_coeff(N_use, C, foldername)
#vector_list = generate_re_ortho_space(N_use, foldername)
H_reduced_non_rotho = generate_reduced_H_non_ortho(N_use, foldername, mpo_ref)
W = get_W(N_use, foldername)
coeff = get_S(W)
coeff = np.array(coeff)
H_reduced = np.einsum('ik, kl, jl -> ij', coeff.conj(), H_reduced_non_rotho, coeff)
H_reduced = remain_only_tridiagonal_elements(H_reduced)

After achieving Krylov vectors, we implement Lanczos algorithm using these Krylov vectors. In this notebook, all the expectation values are calculated using ED for convenience. Since calculating the expectation value (contracting the tensor network) doesn't bring new errors, it makes no difference whether we use tensor network or ED to calculate the matrix elements for reduced Hamiltonian.

In [ ]:
e_ritz, v_ritz = ptn.global_krylov_method.solve_ritz(foldername, H_reduced, N_Krylov, coeff, max_bond_Krylov, e_ground, mpo_ref)